<h1>It is a Coursera Capstone project:</h1>

In [1]:
print('Hello Capstone Project Course!') 

Hello Capstone Project Course!


<h1>Postalcodes from Wikipedia</h1>

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url='https://en.m.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

res = requests.get(url)
res = BeautifulSoup(res.content,'lxml')
table = res.find('table')
df=pd.read_html(str(table))[0]

In [4]:
df.columns=['PostalCode', 'Borough', 'Neighborhood'] 
df=df[df['Borough']!='Not assigned']
df.loc[df['Neighborhood']=='Not assigned','Neighborhood']=df['Borough']
df['Neighborhood']=df['Neighborhood'].str.replace(' /', ',')
df.reset_index(drop=True, inplace=True)

In [5]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df.shape

(103, 3)

<h1>Geolocation data</h1>

In [7]:
locdf=pd.read_csv('https://cocl.us/Geospatial_data')
locdf.columns=['PostalCode', 'Latitude', 'Longitude']
df=df.join(locdf.set_index('PostalCode'), on='PostalCode')

df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h1>Foursquare Venues</h1>

In [8]:
filter=[('Toronto' in bor)  for bor in df['Borough']]
toronto=df[filter].reset_index(drop=True)
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [9]:
toronto.shape

(39, 5)

In [10]:
#!conda install -c conda-forge geopy --yes
!pip install geopy
from geopy.geocoders import Nominatim

In [11]:
address='Toronto'
geolocator=Nominatim(user_agent='tor_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [12]:
#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium

In [13]:
map=folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lon, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood']):
  label=folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lon], 
      radius=5,
      color='blue',
      popup=label,
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7,
      parse_html=False).add_to(map)

map

In [14]:
CLIENT_ID='YOUR_ID'
CLIENT_SECRET='YOUR_SECRET'
VERSION='20180605'
LIMIT=100
RADIUS=500

In [15]:
# The code was removed by Watson Studio for sharing.

In [16]:
def getNearbyVenues(names, lats, lons):
    list=[]
    for n, la, lo in zip(names, lats, lons):
        url= 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, la, lo, RADIUS, LIMIT)
        results= requests.get(url).json()['response']['groups'][0]['items']
        list.append([(
         n, 
         la, 
         lo, 
         v['venue']['name'], 
         v['venue']['location']['lat'], 
         v['venue']['location']['lng'], 
         v['venue']['categories'][0]['name']) for v in results]
        )
    
    nearby=pd.DataFrame([item for vlist in list for item in vlist])
    nearby.columns=[
     'Neighborhood',
     'Neighborhood Latitude', 
     'Neighborhood Longitude',
     'Venue',
     'Venue Latitude',
     'Venue Longitude', 
     'Venue Category'
    ]

    return(nearby)

In [17]:
venues=getNearbyVenues(
  names=toronto['Neighborhood'],
  lats=toronto['Latitude'],
  lons=toronto['Longitude']
)
print(venues.shape)

(1623, 7)


In [18]:
onehot=pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
onehot['Neighborhood']=venues['Neighborhood']
fixed_columns=[onehot.columns[-1]]+list(onehot.columns[:-1])
onehot=onehot[fixed_columns]
grouped=onehot.groupby('Neighborhood').mean().reset_index()
print(grouped.shape)

(39, 231)


In [19]:
def most_common_venues(row, top):
    row_categories=row.iloc[1:]
    row_categories_sorted= row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:top]

In [20]:
import numpy as np

In [21]:
top=10
indicators=['st', 'nd', 'rd']
columns=['Neighborhood']
for ind in np.arange(top):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicator[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

sorted=pd.DataFrame(columns=columns)
sorted['Neighborhood']=grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    sorted.iloc[ind, 1:]=most_common_venues(grouped.iloc[ind, :],top)

sorted.head()

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Café,Cheese Shop,Restaurant,Bistro,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Yoga Studio,Climbing Gym,Stadium,Bakery,Intersection,Burrito Place,Italian Restaurant
2,Business reply mail Processing Centre,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Butcher,Burrito Place,Restaurant,Brewery,Light Rail Station
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Boutique,Rental Car Location,Bar,Plane
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Thai Restaurant,Bubble Tea Shop,Bar,Burger Joint,Dessert Shop,Japanese Restaurant


<h1>Clustering</h1>

In [22]:
from sklearn.cluster import KMeans

In [23]:
clustering=grouped.drop('Neighborhood', 1)

k=5
kmeans=KMeans(n_clusters=k, random_state=0)
kmeans.fit(clustering)

sorted.insert(0,'Cluster Labels', kmeans.labels_)

merged=toronto
merged=merged.join(sorted.set_index('Neighborhood'), on='Neighborhood') 

In [24]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [25]:
map=folium.Map(location=[latitude, longitude], zoom_start=12)

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]

for lat, lon, poi, clus in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label=folium.Popup(str(poi)+' Cluster '+str(clus), parse_html=True)
    folium.CircleMarker(
        [lat, lon], 
        radius=5,
        color=rainbow[clus-1], 
        popup=label,
        fill=True,
        fill_color=rainbow[clus-1], 
        fill_opacity=0.7).add_to(map)
                                                   
map

<h2>Cluster 1</h2>

In [26]:
merged.loc[merged['Cluster Labels']==0, merged.columns[[2]+list(range(k, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Bakery,Park,Pub,Theater,Restaurant,Café,Breakfast Spot,Yoga Studio,Performing Arts Venue
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Juice Bar,Bank,Bar,Café,Diner,Burrito Place
2,"Garden District, Ryerson",0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Restaurant,Middle Eastern Restaurant,Hotel
3,St. James Town,0,Café,Coffee Shop,American Restaurant,Gastropub,Cocktail Bar,Clothing Store,Beer Bar,Restaurant,Gym,Italian Restaurant
4,The Beaches,0,Coffee Shop,Trail,Health Food Store,Pub,College Rec Center,Colombian Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
5,Berczy Park,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Café,Cheese Shop,Restaurant,Bistro,Breakfast Spot
6,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Thai Restaurant,Bubble Tea Shop,Bar,Burger Joint,Dessert Shop,Japanese Restaurant
7,Christie,0,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Diner,Restaurant,Baby Store,Athletics & Sports,Nightclub
8,"Richmond, Adelaide, King",0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Gym,Deli / Bodega,Thai Restaurant,Steakhouse,Salad Place
9,"Dufferin, Dovercourt Village",0,Pharmacy,Bakery,Gas Station,Brewery,Smoke Shop,Liquor Store,Café,Bar,Bank,Pool


<h2>Cluster 2</h2>

In [27]:
merged.loc[merged['Cluster Labels']==1, merged.columns[[2]+list(range(k, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",1,Gym,Restaurant,Women's Store,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


<h2>Cluster 3</h2>

In [28]:
merged.loc[merged['Cluster Labels']==2, merged.columns[[2]+list(range(k, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Forest Hill North & West,2,Park,Trail,Sushi Restaurant,Jewelry Store,College Rec Center,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
33,Rosedale,2,Park,Trail,Playground,Creperie,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


<h2>Cluster 4</h2>

In [29]:
merged.loc[merged['Cluster Labels']==3, merged.columns[[2]+list(range(k, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,3,Garden,Ice Cream Shop,Home Service,Women's Store,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


<h2>Cluster 5</h2>

In [30]:
merged.loc[merged['Cluster Labels']==4, merged.columns[[2]+list(range(k, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,4,Park,Bus Line,Swim School,Ethiopian Restaurant,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


<h2>Clustering summary</h2>
<ul>
    <li>Coffee shop & café</li>
    <li>Gym</li>
    <li>Park & trail</li>
    <li>Garden</li>
    <li>Park & bus</li>
</ul>

There are many coffee shop and café among the most popular locations in the first cluster which is why I named it Caffee shop & café.
In the second cluster there is only one venue and in the first place in the ranking is the gym. In the other clusters, the gym does not have a prominent place. I named it Gym.
There are two locations in the third cluster. Their two most common locations are the park and the trail. Park & trail are called.
The fourth cluster is the only one where the garden is in the first place. I named it Garden.
The fifth cluster is called Park & bus because it is similar to the third cluster, but here the Bus line is the second most common.